<a href="https://colab.research.google.com/github/Alex112525/Neural-Networks-with-TensorFlow/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate https://storage.googleapis.com/platzi-tf2/sign-language-img.zip 
!unzip -q /content/sign-language-img.zip -d Dataset

--2023-02-24 02:06:32--  https://storage.googleapis.com/platzi-tf2/sign-language-img.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 173.194.69.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24265584 (23M) [application/zip]
Saving to: ‘sign-language-img.zip’

sign-language-img.z 100%[===================>]  23.14M  18.8MB/s    in 1.2s    

2023-02-24 02:06:33 (18.8 MB/s) - ‘sign-language-img.zip’ saved [24265584/24265584]



In [21]:
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import models, layers, optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator

#Load Dataset

In [3]:
train_dir = "/content/Dataset/Test"
test_dir = "/content/Dataset/Train"

Build datasets generator

In [4]:
train_datagen = ImageDataGenerator(rescale = 1/255)
test_datagen = ImageDataGenerator(rescale = 1/255, validation_split=0.3)

In [22]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 128,
    shuffle=True,
    class_mode = "categorical",
    color_mode = "rgb",
    subset = "training"
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 128,
    shuffle=True,
    class_mode = "categorical",
    color_mode = "rgb",
    subset = "validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 128,
    shuffle=True,
    class_mode = "categorical",
    color_mode = "rgb"
)

Found 7172 images belonging to 24 classes.
Found 8226 images belonging to 24 classes.
Found 27455 images belonging to 24 classes.


Get classes from dataset

In [6]:
class_names = sorted(os.listdir(train_dir))
n_classes = len(class_names)
print(class_names," Classes:", n_classes)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']  Classes: 24


#Transfer learning
Is a technique that involves taking a pre-trained model, typically trained on a large dataset, and using it as a starting point for a new task with a smaller dataset. This is often better than training a new model from scratch because the pre-trained model has already learned a lot of useful features and patterns that can be applied to the new task. 

**Transfer learning** can save time and computational resources and can lead to better performance on the new task, especially when the new dataset is small or similar to the original dataset used to train the pre-trained model.

In [9]:
from keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(include_top=False,
                                input_tensor=layers.Input(shape=(150, 150, 3)))

for layer in pre_trained_model.layers:
  layer.trainable = False

**Inception v3** is a convolutional neural network architecture developed by Google. It is designed to improve the accuracy and efficiency of image classification tasks.

The key innovation in the Inception v3 architecture is the use of **"Inception modules"** which are designed to capture multi-scale features by performing multiple convolution operations in parallel. These modules use 1x1, 3x3, and 5x5 convolutions, as well as pooling, to capture features at different spatial scales.

Overall, Inception v3 is a powerful and efficient architecture that has been widely used for a variety of image classification tasks.

In [10]:
last_layer = pre_trained_model.get_layer("mixed7")
last_output = last_layer.output

**Keras functional** API is a way to build complex models in Keras that are not sequential. 

It allows for the creation of multi-input and multi-output models, models with shared layers, and models with non-sequential data flows. Instead of building models as a sequence of layers, the functional API builds graphs of layers, allowing for more flexibility in the architecture of the model.

In [12]:
out = layers.Flatten()(last_output)
out = layers.Dense(128, activation="relu")(out)
out = layers.Dropout(0.2)(out)
out = layers.Dense(n_classes, activation="softmax")(out)

In [14]:
final_model = tf.keras.Model(pre_trained_model.input, out)
final_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_145 (Conv2D)            (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_145 (Batch  (None, 74, 74, 32)  96          ['conv2d_145[0][0]']             
 Normalization)                                                                                   
                                                                                            

##Training

In [16]:
Model = final_model.fit(train_generator,
                        epochs = 5,
                        validation_data = validation_generator)

Epoch 1/5
57/57 [==============================] - 33s 357ms/step - loss: 0.4746 - accuracy: 0.8755 - val_loss: 0.5054 - val_accuracy: 0.8620
Epoch 2/5
57/57 [==============================] - 17s 301ms/step - loss: 0.0061 - accuracy: 0.9994 - val_loss: 0.5523 - val_accuracy: 0.8542
Epoch 3/5
57/57 [==============================] - 18s 310ms/step - loss: 0.0037 - accuracy: 0.9996 - val_loss: 0.5443 - val_accuracy: 0.8607
Epoch 4/5
57/57 [==============================] - 18s 318ms/step - loss: 0.0030 - accuracy: 0.9993 - val_loss: 0.5144 - val_accuracy: 0.8691
Epoch 5/5
57/57 [==============================] - 17s 308ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.5295 - val_accuracy: 0.8662


##Test

In [20]:
final_model.evaluate(test_generator)

215/215 [==============================] - 33s 152ms/step - loss: 0.5158 - accuracy: 0.8680


[0.5157809853553772, 0.86800217628479]